# Install deequ

In [1]:
import $ivy.`com.amazon.deequ:deequ:2.0.1-spark-3.2`

import $ivy.$                                       

## spark setup

In [2]:
import org.apache.log4j.Logger
import org.apache.log4j.Level

Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("akka").setLevel(Level.OFF)

import org.apache.log4j.Logger

import org.apache.log4j.Level



In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder()
      .master("local")
      .appName("test")
      .config("spark.ui.enabled", "false")
      .getOrCreate()
    spark.sparkContext.setCheckpointDir(System.getProperty("java.io.tmpdir"))
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql.SparkSession

spark: SparkSession = org.apache.spark.sql.SparkSession@21a7a334
import spark.implicits._

In [4]:
import org.apache.spark.sql.DataFrame
import almond.interpreter.api.DisplayData

def show(df:DataFrame) {
    val header = df.columns.map( columnName => s"<td><b>${columnName.toString}<b></td>").mkString("")
    val tableRows = df.collect.map(r => {
        r.toSeq.map(value =>s"<td>$value</td>").mkString("")
    }).map(trow=> s"<tr>$trow<tr>").mkString("")
    val htmlTable = s"<html><table><tr>${header}<tr>$tableRows</table></html>"

    display(DisplayData(
    Map(
      "text/html" -> htmlTable)))
}

import org.apache.spark.sql.DataFrame

import almond.interpreter.api.DisplayData


defined function show

## read data

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv

- instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit :
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [34]:
import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}
val df: DataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("data/day.csv")
df.printSchema
show(df.limit(30))

root
 |-- instant: integer (nullable = true)
 |-- dteday: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- yr: integer (nullable = true)
 |-- mnth: integer (nullable = true)
 |-- holiday: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- workingday: integer (nullable = true)
 |-- weathersit: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- casual: integer (nullable = true)
 |-- registered: integer (nullable = true)
 |-- cnt: integer (nullable = true)



instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
4,2011-01-04,1,0,1,0,2,1,1,0.2,0.212122,0.590435,0.160296,108,1454,1562
5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.22927,0.436957,0.1869,82,1518,1600
6,2011-01-06,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,88,1518,1606
7,2011-01-07,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510
8,2011-01-08,1,0,1,0,6,0,2,0.165,0.162254,0.535833,0.266804,68,891,959
9,2011-01-09,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.36195,54,768,822
10,2011-01-10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321


import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}

df: DataFrame = [instant: int, dteday: string ... 14 more fields]

# Verification suites

## where condition

In [33]:
import com.amazon.deequ.{VerificationResult, VerificationSuite}
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.constraints.{ConstrainableDataTypes, ConstraintStatus}
import org.apache.spark.sql.DataFrame

val verificationResult = VerificationSuite()
      .onData(df.filter('yr===1 && 'mnth === 2))
      .addCheck(
        Check(CheckLevel.Error, "where")
            .isContainedIn("weekday", Array("6","0")).where("workingday = 0 and holiday != 1")
            .satisfies("casual/registered > 0.1", "check casual percentage",_>=0.7)
            .satisfies("casual/registered > 0.1", "check casual precentage in weekends", _>=0.7).where("workingday=0")
          ).run
val dfResults = VerificationResult.checkResultsAsDataFrame(spark, verificationResult)
show(dfResults.orderBy("constraint_status"))

check,check_level,check_status,constraint,constraint_status,constraint_message
where,Error,Error,"ComplianceConstraint(Compliance(check casual percentage,casual/registered > 0.1,None))",Failure,Value: 0.27586206896551724 does not meet the constraint requirement!
where,Error,Error,"ComplianceConstraint(Compliance(weekday contained in 6,0,`weekday` IS NULL OR `weekday` IN ('6','0'),Some(workingday = 0 and holiday != 1)))",Success,
where,Error,Error,"ComplianceConstraint(Compliance(check casual precentage in weekends,casual/registered > 0.1,Some(workingday=0)))",Success,


import com.amazon.deequ.{VerificationResult, VerificationSuite}

import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}

import com.amazon.deequ.constraints.{ConstrainableDataTypes, ConstraintStatus}

import org.apache.spark.sql.DataFrame


verificationResult: VerificationResult = VerificationResult(
  Error,
  Map(
    Check(
      Error,
      "where",
      List(
        ComplianceConstraint(Compliance(weekday contained in 6,0,`weekday` IS NULL OR `weekday` IN ('6','0'),Some(workingday = 0 and holiday != 1))),
        ComplianceConstraint(Compliance(check casual percentage,casual/registered > 0.1,None)),
        ComplianceConstraint(Compliance(check casual precentage in weekends,casual/registered > 0.1,Some(workingday=0)))
      )
    ) -> CheckResult(
      Check(
        Error,
        "where",
        List(
          ComplianceConstraint(Compliance(weekday contained in 6,0,`weekday` IS NULL OR `weekday` IN ('6','0'),Some(workingday = 0 and holiday != 1))),
          Comp

## Hints

In [39]:
 val verificationResult = VerificationSuite()
      .onData(df.filter("yr=1"))
      .addCheck(Check(CheckLevel.Error, "With errors")
        // not satisfied constraints
        .isUnique("temp").where("workingday='1'")
        .hasDistinctness(Seq("temp"), _ == 1)
        .hasDistinctness(Seq("instant"), _ < 1, Some("Check for instant uniqueness"))
        .hasDataType("temp", ConstrainableDataTypes.String, _==1, Some("Check are there string data"))
        .hasSize(_ == 365, Some("Expected size 365"))
        .hasSum("cnt", _ > 10000000, hint = Some("Expected bike shares cnt > 10000000"))
        .hasCorrelation("season", "temp", _ > 0.5)
        .hasCorrelation("temp", "cnt", _ < 0.5)
        .satisfies(columnCondition = "temp > hum",
          constraintName = "temp check in workday",
          assertion = _ > 0.5)
      ).run

    val dfResults = VerificationResult.checkResultsAsDataFrame(spark, verificationResult)
    show(dfResults.orderBy("constraint_status"))

check,check_level,check_status,constraint,constraint_status,constraint_message
With errors,Error,Error,"UniquenessConstraint(Uniqueness(List(temp),Some(workingday='1')))",Failure,Value: 0.748 does not meet the constraint requirement!
With errors,Error,Error,"DistinctnessConstraint(Distinctness(List(temp),None))",Failure,Value: 0.8005464480874317 does not meet the constraint requirement!
With errors,Error,Error,"DistinctnessConstraint(Distinctness(List(instant),None))",Failure,Value: 1.0 does not meet the constraint requirement! Check for instant uniqueness
With errors,Error,Error,"AnalysisBasedConstraint(DataType(temp,None),,Some(),Some(Check are there string data))",Failure,Value: 0.0 does not meet the constraint requirement! Check are there string data
With errors,Error,Error,SizeConstraint(Size(None)),Failure,Value: 366 does not meet the constraint requirement! Expected size 365
With errors,Error,Error,"SumConstraint(Sum(cnt,None))",Failure,Value: 2049576.0 does not meet the constraint requirement! Expected bike shares cnt > 10000000
With errors,Error,Error,"CorrelationConstraint(Correlation(season,temp,None))",Failure,Value: 0.29387642123889274 does not meet the constraint requirement!
With errors,Error,Error,"CorrelationConstraint(Correlation(temp,cnt,None))",Failure,Value: 0.7137931988838034 does not meet the constraint requirement!
With errors,Error,Error,"ComplianceConstraint(Compliance(temp check in workday,temp > hum,None))",Failure,Value: 0.31420765027322406 does not meet the constraint requirement!


verificationResult: VerificationResult = VerificationResult(
  Error,
  Map(
    Check(
      Error,
      "With errors",
      List(
        UniquenessConstraint(Uniqueness(List(temp),Some(workingday='1'))),
        DistinctnessConstraint(Distinctness(List(temp),None)),
        DistinctnessConstraint(Distinctness(List(instant),None)),
        AnalysisBasedConstraint(
          DataType("temp", None),
          ammonite.$sess.cmd38$Helper$$Lambda$6468/1423195614@282a6f02,
          Some(
            com.amazon.deequ.constraints.Constraint$$$Lambda$6156/755216664@5acffcfe
          ),
          Some("Check are there string data")
        ),
        SizeConstraint(Size(None)),
        SumConstraint(Sum(cnt,None)),
        CorrelationConstraint(Correlation(season,temp,None)),
        CorrelationConstraint(Correlation(temp,cnt,None)),
        ComplianceConstraint(Compliance(temp check in workday,temp > hum,None))
      )
    ) -> CheckResult(
      Check(
        Error,
        "With error